

- From Mattman to another student on slack
    - Don't build the xml file, just use the docker
    - Run this: docker run -it -p 8764:8764 uscdatascience/inception-rest-tika
    - This is how to test it: http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://upload.wikimedia.org/wikipedia/commons/f/f6/Working_Dogs%2C_Handlers_Share_Special_Bond_DVIDS124942.jpg
    - more documentation here: https://cwiki.apache.org/confluence/display/TIKA/TikaAndVision#TikaAndVision-a.Usingdocker(Recommended)
    - to find image caption, use this docker: docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika
    - see this: https://github.com/USCDataScience/tika-dockers
    - then see this: http://localhost:8764/inception/v3/caption/image?url=https://upload.wikimedia.org/w[…]s_Thames_Tigers_2007.jpg/1200px-Marcus_Thames_Tigers_2007.jpg
    - additional documentation: https://cwiki.apache.org/confluence/display/TIKA/ImageCaption#ImageCaption-a.Usingdocker(Recommended)


# Follow these steps to get started:

Check out this if you get stuck: https://cwiki.apache.org/confluence/display/TIKA/TikaAndVision#TikaAndVision-a.Usingdocker(Recommended)

- install the docker desktop app
- pip install tensorflow
- git clone https://github.com/tensorflow/models.git
- export PYTHONPATH="$PYTHONPATH:`pwd`:`pwd`/slim"
- python -c 'import tensorflow, numpy, dataset; print("OK")'
- git clone https://github.com/USCDataScience/tika-dockers.git && cd tika-dockers
- docker build -f InceptionRestDockerfile -t uscdatascience/inception-rest-tika .
- docker run -p 8764:8764 -it uscdatascience/inception-rest-tika 
- Below is what Mattman said to do in slack. Note that the docker run command is very similar to the one we just did. I believe they are interchangable commands. 
- docker run -it -p 8764:8764 uscdatascience/inception-rest-tika
- to test it, plug this into web browser: http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://upload.wikimedia.org/wikipedia/commons/f/f6/Working_Dogs%2C_Handlers_Share_Special_Bond_DVIDS124942.jpg 
- if the test doesn't work, Go to docker desktop and clear out: 1) any unused or old containers; and 2) delete your image for uscdatascience/inception-rest-tika. Try again. 

In order to get a url that points to your downloaded photos, do the follwing:
- "python -mhttp.server 8888" in the directory where your photos are
- Follow this format to point to a specific photo http://localhost:8888/Pixstory-image-161668402049563.jpg
- here's another image to try: http://localhost:8888/Pixstory-image-162521751234631.jpeg
- if you want to see the list of links to images: http://localhost:8888



# URL Pointing to Downloaded Images STEPS 

In order to get a url that points to your downloaded photos, do the follwing:
- "python -mhttp.server 8888" in the directory where your photos are
- Follow this format to point to a specific photo http://localhost:8888/Pixstory-image-161668402049563.jpg
- here's another image to try: http://localhost:8888/Pixstory-image-162521751234631.jpeg
- if you want to see the list of links to images: http://localhost:8888


This IP addressed worked at school: http://10.25.179.208:8888 AKA <your_local_ip>:8888.

This IP address worked at home: http://192.168.1.13:8888

Steps to do this: 

- Get IP address of your local machine: ipconfig (Windows) or ifconfig (Mac or Linux). 





In [1]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd



In [12]:
images_url_usc  = 'http://10.25.179.208:8888'

images_url_home = 'http://192.168.1.13:8888'

# Running Tests

# Detect Object Tests

In [3]:
#docker run -it -p 8764:8764 uscdatascience/inception-rest-tika

#testing this with a pixtosry image online - it works! Even when the test link doesn't work.... I wonder why


# Load the CSV file into a pandas DataFrame
df = pd.read_csv('../Datasets/Master_Dataset_Raw.csv')


# Define a lambda function to modify the URLs
modify_url = lambda url: url.replace(".com/", ".com/optimized/")

# Apply the lambda function to the "Media" column of the DataFrame
df['Media'] = df['Media'].apply(modify_url)

first_media = df.loc[0, 'Media']
print(first_media)


#link is https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg
#modified link is 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg'

url3 = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg'

#another link is 

response = requests.get(url3)
json_response = response.json()

print(json_response)



https://image.pixstory.com/optimized/Pixstory-image-165942680078373.jpeg


JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.</p>
: 0

In [11]:
#Testing with downloaded image

#Example image URL "http://10.25.179.208:8888/Pixstory-image-164318304274781.png"

#Modified URL skeleton format 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=<URL>'

 #this one works!!!!!
url3_usc = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=http://10.25.179.208:8888/Pixstory-image-164318304274781.png'

url3 = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png'

response = requests.get(url3)
json_response = response.json()

print(json_response)


JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.</p>
: 0

# Generated Image Caption Tests

In [4]:
#docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika

#test link: http://localhost:8764/inception/v3/caption/image?url=https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/Marcus_Thames_Tigers_2007.jpg/1200px-Marcus_Thames_Tigers_2007.jpg

#skeleton modified URL: http://localhost:8764/inception/v3/caption/image?url=<URL>

url_usc = 'http://localhost:8764/inception/v3/caption/image?url=http://10.25.179.208:8888/Pixstory-image-164318304274781.png'
url3 = 'http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png'

response = requests.get(url3)
json_response = response.json()

print(json_response)

# Extract the first listed sentence
first_sentence = json_response['captions'][0]['sentence']

print(first_sentence)


{'captions': [{'confidence': 3.2656944553199396e-05, 'sentence': 'an old fashioned clock sitting on top of a table .'}, {'confidence': 2.4475188480119203e-05, 'sentence': 'an old fashioned clock sitting in the middle of a field .'}, {'confidence': 1.5979679814940473e-05, 'sentence': 'an old fashioned clock sitting on top of a wooden table .'}], 'beam_size': 3, 'max_caption_length': 20, 'time': {'read': 31, 'captioning': 1250, 'units': 'ms'}}
an old fashioned clock sitting on top of a table .


# Image Analysis Starts Here

# DF of Detect Object Using a List of URLs

In [8]:
# This works!

# List of image URLs
image_urls = [
    'http://192.168.1.13:8888/Pixstory-image-164318304274781.png',
    'http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg',
    'http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg'

]
# List to store the classification results
results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api_url = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=' + url

#http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png

#http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png

    # Send the API request
    response = requests.get(api_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the top two labels and their confidences from the JSON response
    classnames = json_response['classnames']
    confidence = json_response['confidence']


    # Add the results to the list
    results.append({
        'api_url': api_url,
        'url': url,
        'detect object': classnames,
    

    })

# Create a Pandas dataframe from the list of dictionaries
df = pd.DataFrame(results)


df


JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.</p>
: 0

# Df of image captions from list of links

In [10]:
# This works!

# List of image URLs
image_urls = [
    'http://192.168.1.13:8888/Pixstory-image-164318304274781.png',
    'http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg',
    'http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg'

]
# List to store the classification results
results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api2_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api2_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    results.append({
        'api_url2': api2_url,
        'url': url, 
        'caption':first_sentence
    })


# Create a Pandas dataframe from the list of dictionaries
df2 = pd.DataFrame(results)

# Set display options to show all columns and avoid truncation of string data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


df2


,api_url2,url,caption
0,http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png,http://192.168.1.13:8888/Pixstory-image-164318304274781.png,an old fashioned clock sitting on top of a table .
1,http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,a bunch of bananas that are on a table .
2,http://localhost:8764/inception/v3/caption/image?url=http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,a man sitting on a couch with a laptop .


# Webscraping to get list of image URLs

In [14]:
# Creating a list of urls from webpage 
# This works!


webpage_url = images_url_home

response = requests.get(webpage_url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

image_urls = []
for i, link in enumerate(soup.find_all('a')):
    if i >= 10:
        break
    href = link.get('href')
    if href is not None and href.startswith('http'):
        image_urls.append(href)
    elif href is not None:
        image_urls.append(webpage_url + '/' + href)

print(image_urls)


['http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg', 'http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg', 'http://192.168.1.13:8888/Pixstory-image-161984246212221.jpeg', 'http://192.168.1.13:8888/Pixstory-image-162014760674045.jpeg', 'http://192.168.1.13:8888/Pixstory-image-162153257364412.png', 'http://192.168.1.13:8888/Pixstory-image-162157341332029.jpeg', 'http://192.168.1.13:8888/Pixstory-image-162244448366044.jpeg', 'http://192.168.1.13:8888/Pixstory-image-16233178221386.jpeg', 'http://192.168.1.13:8888/Pixstory-image-162347005942405.jpeg', 'http://192.168.1.13:8888/Pixstory-image-162454466467149.jpeg']


# Df of image captions through web scraping

In [15]:


captions_results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api2_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api2_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    captions_results.append({

        'url': url, 
        'caption':first_sentence
    })

# Create a Pandas dataframe from the list of dictionaries
df2 = pd.DataFrame(captions_results)

# Set display options to show all columns and avoid truncation of string data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Display the dataframe
df2

,url,caption
0,http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,a bunch of bananas that are on a table .
1,http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,a man sitting on a couch with a laptop .
2,http://192.168.1.13:8888/Pixstory-image-161984246212221.jpeg,a man in a suit and tie holding a cell phone .
3,http://192.168.1.13:8888/Pixstory-image-162014760674045.jpeg,a bunch of birds that are flying in the air .
4,http://192.168.1.13:8888/Pixstory-image-162153257364412.png,a bike parked on the side of a road .
5,http://192.168.1.13:8888/Pixstory-image-162157341332029.jpeg,a man in a suit and tie standing in a room .
6,http://192.168.1.13:8888/Pixstory-image-162244448366044.jpeg,a bunch of flowers that are on a table .
7,http://192.168.1.13:8888/Pixstory-image-16233178221386.jpeg,a man in a suit and tie standing in a room .
8,http://192.168.1.13:8888/Pixstory-image-162347005942405.jpeg,a close up of a vase with flowers in it
9,http://192.168.1.13:8888/Pixstory-image-162454466467149.jpeg,a cat laying on top of a blanket on a bed .


# Df of detect objects through webscraping 

In [ ]:

objects_results = []

# Loop through the image URLs and classify each image
for url in image_urls:


        # Construct the API URL for the image classification
    api_url = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=' + url

    # Send the API request
    response = requests.get(api_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    captions_results.append({

        'url': url, 
        'caption':first_sentence
    })

# Create a Pandas dataframe from the list of dictionaries
df2 = pd.DataFrame(captions_results)

# Set display options to show all columns and avoid truncation of string data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Display the dataframe
df2